In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('./../token.json'):
        creds = Credentials.from_authorized_user_file('./../token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                './../credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('./../token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

In [3]:
def get_gcal_events(creds, max_events=10):
    """Gets the upcoming 10 events from the user's Google Calendar.

    Args:
        creds: Credentials, the user's valid credentials.

    Returns:
        List of dicts, the upcoming 10 events on the user's calendar.
    """
    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        print('Getting the upcoming events')
        events_result = service.events().list(calendarId='primary', timeMin=now,
                                              maxResults=max_events, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return []

        return events

    except HttpError as error:
        print('An error occurred: %s' % error)
        return []


In [4]:
import pprint as pp

pp.pprint(get_gcal_events(creds=get_credentials()))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=816563402707-irbvafirb1b3qlla0vidg9dc0q36768k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56411%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=oRRsgsyRxRHX77UzTFnwp7tYT53E8l&access_type=offline
Getting the upcoming events
[{'created': '2023-09-23T23:03:52.000Z',
  'creator': {'email': 'aps.maildrop@gmail.com', 'self': True},
  'end': {'dateTime': '2023-10-07T20:30:00-04:00',
          'timeZone': 'America/New_York'},
  'etag': '"3391025587658000"',
  'eventType': 'default',
  'guestsCanInviteOthers': False,
  'guestsCanSeeOtherGuests': False,
  'htmlLink': 'https://www.google.com/calendar/event?eid=NTAxY3I3MXMwNWRvZDhyZTk4dGluaDhza21fMjAyMzEwMDdUMTcwMDAwWiBhcHMubWFpbGRyb3BAbQ',
  'iCalUID': '501cr71s05dod8re98tinh8skm@google.com',
  'id': '501cr71s05dod8re98tinh8skm_20231007T170000Z',
  'kind': 'calendar#event',
  'organizer

In [5]:
def create_gcal_event(creds, event):
    """Creates an event on the user's Google Calendar.

    Args:
        creds: Credentials, the user's valid credentials.
        event: Dict, the event to be created.

    Returns:
        Dict, the created event.
    """
    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        event = service.events().insert(calendarId='primary', body=event).execute()
        print('Event created: %s' % (event.get('htmlLink')))
        return event

    except HttpError as error:
        print('An error occurred: %s' % error)
        return None

In [6]:
def get_nyt_now():
    """Gets the current time in New York.

    Returns:
        Dict, the current time in New York in the format {'dateTime': 'YYYY-MM-DDTHH:MM:SS-05:00', 'timeZone': 'America/New_York'}.
    """
    nyt_now = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=-5)))
    nyt_now_str = nyt_now.strftime('%Y-%m-%dT%H:%M:%S-05:00')
    return {'dateTime': nyt_now_str, 'timeZone': 'America/New_York'}


In [7]:
get_nyt_now()

{'dateTime': '2023-10-07T17:16:25-05:00', 'timeZone': 'America/New_York'}

In [8]:
def add_time_to_now(delta):
    """Adds a time delta to the current time in New York.

    Args:
        delta: datetime.timedelta, the time delta to add to the current time.

    Returns:
        Dict, the current time plus the delta in the format {'dateTime': 'YYYY-MM-DDTHH:MM:SS-05:00', 'timeZone': 'America/New_York'}.
    """
    nyt_now = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=-5)))
    nyt_now_plus_delta = nyt_now + delta
    nyt_now_plus_delta_str = nyt_now_plus_delta.strftime('%Y-%m-%dT%H:%M:%S-05:00')
    return {'dateTime': nyt_now_plus_delta_str, 'timeZone': 'America/New_York'}


In [9]:
delta = datetime.timedelta(hours=1)
add_time_to_now(delta=datetime.timedelta(hours=1))

{'dateTime': '2023-10-07T18:16:25-05:00', 'timeZone': 'America/New_York'}

In [10]:
def get_calendar_list(creds, calendar_id='primary'):
    """Calls the Google Calendar API to retrieve the specified calendar list.

    Args:
        creds: Credentials, the user's valid credentials.
        calendar_id: str, the ID of the calendar to retrieve. Defaults to 'primary'.

    Returns:
        Dict, the calendar list.
    """
    try:
        service = build('calendar', 'v3', credentials=creds)

        # Call the Calendar API
        calendar_list = service.calendarList().get(calendarId=calendar_id).execute()
        return calendar_list

    except HttpError as error:
        print('An error occurred: %s' % error)
        return None

In [11]:
get_calendar_list(creds=get_credentials())

{'kind': 'calendar#calendarListEntry',
 'etag': '"1695841878307000"',
 'id': 'aps.maildrop@gmail.com',
 'summary': 'Abhay @ Personal',
 'timeZone': 'America/New_York',
 'colorId': '9',
 'backgroundColor': '#7bd148',
 'foregroundColor': '#000000',
 'selected': True,
 'accessRole': 'owner',
 'defaultReminders': [{'method': 'popup', 'minutes': 30}],
 'notificationSettings': {'notifications': [{'type': 'eventResponse',
    'method': 'email'}]},
 'primary': True,
 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

In [12]:
# Refer to the Python quickstart on how to setup the environment:
# https://developers.google.com/calendar/quickstart/python
# Change the scope to 'https://www.googleapis.com/auth/calendar' and delete any
# stored credentials.

event = {
  'summary': 'Test event create',
  'location': '',
  'description': 'test',
  'start': get_nyt_now(),
  'end': add_time_to_now(delta=datetime.timedelta(hours=1)),
  'recurrence': [
    # 'RRULE:FREQ=DAILY;COUNT=2'
  ],
  'attendees': [],
  'reminders': {
    # 'useDefault': False,
    # 'overrides': [
    #   # {'method': 'email', 'minutes': 24 * 60},
    #   {'method': 'popup', 'minutes': 10},
    # ],
  },
}


event = create_gcal_event(creds=get_credentials(), event=event)
print('Event created: %s' % (event.get('htmlLink')))


Event created: https://www.google.com/calendar/event?eid=M3FrYmVpcDBpNjVyajNwMmhtZWd2cGdhMXMgYXBzLm1haWxkcm9wQG0
Event created: https://www.google.com/calendar/event?eid=M3FrYmVpcDBpNjVyajNwMmhtZWd2cGdhMXMgYXBzLm1haWxkcm9wQG0
